In [ ]:
# baseline model


# 4 ways to improve the model

# 1. Increase training data size

# 2. Add more features to the training data

# 3. Try different algorithms

# 4. Tune the hyperparameters of the model



In [8]:
import pandas as pd

# Load the data
data = pd.read_parquet('../data/transformed/yellow_tripdata_features_target.parquet')

In [9]:
data.head()

data[data['PULocationID'] == 43].head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,pickup_hour,PULocationID,target
604417,273.0,262.0,79.0,22.0,11.0,11.0,11.0,32.0,28.0,67.0,...,176.0,138.0,90.0,58.0,43.0,48.0,18.0,2019-01-02 01:00:00,43,5.0
604418,262.0,79.0,22.0,11.0,11.0,11.0,32.0,28.0,67.0,87.0,...,138.0,90.0,58.0,43.0,48.0,18.0,17.0,2019-01-02 02:00:00,43,2.0
604419,79.0,22.0,11.0,11.0,11.0,32.0,28.0,67.0,87.0,126.0,...,90.0,58.0,43.0,48.0,18.0,17.0,5.0,2019-01-02 03:00:00,43,3.0
604420,22.0,11.0,11.0,11.0,32.0,28.0,67.0,87.0,126.0,171.0,...,58.0,43.0,48.0,18.0,17.0,5.0,2.0,2019-01-02 04:00:00,43,8.0
604421,11.0,11.0,11.0,32.0,28.0,67.0,87.0,126.0,171.0,252.0,...,43.0,48.0,18.0,17.0,5.0,2.0,3.0,2019-01-02 05:00:00,43,14.0


In [23]:
from datetime import datetime
from typing import Tuple
import pandas as pd

def create_training_sets(PATH, cutoff_date, target_column_name): 
    """
    Generate separate training and testing sets for each PULocationID.
    
    :param PATH: Path to the data file.
    :param cutoff_date: The cutoff date for splitting into train/test sets.
    :param target_column_name: The name of the target column.
    :return: A dictionary where each PULocationID has its (X_train, X_test, y_train, y_test).
    """
    data = pd.read_parquet(PATH)
    
    # Dictionary to store training/testing sets for each PULocationID
    training_sets = {}

    for pulocation_id in data['PULocationID'].unique():
        print(f"Processing PULocationID: {pulocation_id}")
        
        # Filter data specific to each PULocationID
        location_data = data[data['PULocationID'] == pulocation_id].copy()
        location_data.drop(columns=['PULocationID'], inplace=True)
        # Perform train/test split for this location
        X_train, X_test, y_train, y_test = train_test_split(
            location_data,
            cutoff_date,
            target_column_name
        )

        # Store the result in the dictionary
        training_sets[pulocation_id] = (X_train, X_test, y_train, y_test)

    return training_sets


In [24]:
# test the function
training_sets = create_training_sets(
    '../data/transformed/yellow_tripdata_features_target.parquet',
    datetime(2019, 6, 1),
    'target'
)

Processing PULocationID: 4
Processing PULocationID: 7
Processing PULocationID: 10
Processing PULocationID: 11
Processing PULocationID: 12
Processing PULocationID: 13
Processing PULocationID: 14
Processing PULocationID: 17
Processing PULocationID: 18
Processing PULocationID: 19
Processing PULocationID: 20
Processing PULocationID: 21
Processing PULocationID: 24
Processing PULocationID: 25
Processing PULocationID: 33
Processing PULocationID: 34
Processing PULocationID: 35
Processing PULocationID: 36
Processing PULocationID: 37
Processing PULocationID: 39
Processing PULocationID: 40
Processing PULocationID: 41
Processing PULocationID: 42
Processing PULocationID: 43
Processing PULocationID: 45
Processing PULocationID: 48
Processing PULocationID: 49
Processing PULocationID: 50
Processing PULocationID: 51
Processing PULocationID: 52
Processing PULocationID: 54
Processing PULocationID: 56
Processing PULocationID: 60
Processing PULocationID: 61
Processing PULocationID: 63
Processing PULocationI

In [ ]:
# for each pulocationid, we have a tuple of X_train, X_test, y_train, y_test
# now, we can train a model for each pulocationid
# we should create a function to train a model for each pulocationid

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split

def train_model(model, X_train, y_train):
    """
    Train a machine learning model.
    
    :param X_train: The training features.
    :param y_train: The training target.
    :return: The trained model.
    """
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_test, y_test):
    """
    Evaluate a machine learning model.
    
    :param model: The trained model.
    :param X_test: The testing features.
    :param y_test: The testing target.
    :return: The evaluation metric.
    """
    y_pred = model.predict(X_test)
    return root_mean_squared_error(y_test, y_pred)
